In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np

import timeit
from Preprocessing.data_norm import *
from Preprocessing.make_idx import *
from Preprocessing.make_team import *
from Preprocessing.train import *
import re
import copy as cp

import pandas as pd

In [13]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial
# the default is gpu but if gpu is not available,then use cpu
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
device = torch.device('cpu')
print_every = 100
# show which device to use
print('using device:', device)

using device: cpu


In [14]:
#Load testing odding file name
year='2018'
prefix='bettingOdds'
suffix='.csv'
bettingFileName=prefix+year+suffix

In [15]:
teamName = ["Atlanta Hawks","Boston Celtics","Brooklyn Nets","Charlotte Bobcats","Charlotte Hornets",
"Chicago Bulls","Cleveland Cavaliers","Dallas Mavericks","Denver Nuggets","Detroit Pistons",
"Golden State Warriors","Houston Rockets","Indiana Pacers","Los Angeles Clippers","Los Angeles Lakers",
"Memphis Grizzlies","Miami Heat","Milwaukee Bucks","Minnesota Timberwolves","New Jeresy Nets",
"New Orleans Hornets","New Orleans Pelicans","New York Knicks","Oklahoma City Thunder","Orlando Magic",
"Philadelphia 76ers","Phoenix Suns","Portland Trail Blazers","Sacramento Kings","San Antonio Spurs",
"Seattle SuperSonics","Toronto Raptors","Utah Jazz","Vancouver Grizzlies","Washington Bullets","Washington Wizards"]

Abbr = ["ATL","BOS","BKN","CHA","CHA","CHI","CLE","DAL","DEN","DET","GSW","HOU","IND","LAC","LAL","MEM",
"MIA","MIL","MIN","NJN","NOH","NOP","NYK","OKC","ORL","PHI","PHO","POR","SAC","SAS","SEA","TOR",
"UTA","VAN","WSB","WAS"]


In [16]:
'''
when use different idx,we use different features to do training
note:!when saving the model, we should tell which idx used
'''

#index_code1
need_idx = [1,2,3,4,5,6,7,8,9,10,11,12,17,18,19,20]
#need_idx = [2,3,4,5,6,7,8,9,10,11,12,17,18,19]
#need_idx = [1]

In [20]:
# '''
#25 years training, 4 validation, 1 testing
trainYear = 25
startYear_train = 1989
validationYear = 4
startYear_validation = startYear_train + trainYear
testYear = 1
startYear_test = startYear_validation + validationYear
# #initialization
# player_list_train = [0] * trainYear
# winrate_list_train = [0] * trainYear
# data_list_train = [0] * trainYear
# team_idx_train = [0] * trainYear
# data_input_train = [0] * trainYear
# loader_train = {}
# loader_validation = {}
# loader_test = {}
# for index in range(trainYear):
#     # read the statics and winrate
#     playerstats_train = "Player_Data\AvgPlayerStats" + str(index+startYear_train) + ".csv"
#     teamstats_train = "Team_data\TeamStats" + str(index+startYear_train) + ".csv"
#     # change them into numpy
#     player_list_train[index] = pd.read_csv(playerstats_train).values
#     winrate_list_train[index] = pd.read_csv(teamstats_train).values
#     # we throw away the players name,team ,age and so on
#     data_list_train[index] = player_list_train[index][:,4:]
#     # do normalization
#     data_list_train[index] = data_norm(data_list_train[index])   
#     # make a index which is abbr
#     team_idx_train[index] = make_idx(winrate_list_train[index],Abbr,teamName)
#     # put players in a same team into one matrix
#     data_input_train[index] = make_team(player_list_train[index],data_list_train[index],team_idx_train[index],str(index+startYear_train),device,dtype,need_idx)
#     # put everyyears' data into one dictionary
#     loader_train = dict(loader_train,**data_input_train[index])
    
# player_list_validation = [0] * validationYear
# winrate_list_validation = [0] * validationYear
# data_list_validation = [0] * validationYear
# team_idx_validation = [0] * validationYear
# data_input_validation = [0] * validationYear
# for index in range(validationYear):
#     playerstats_validation = "Player_Data\AvgPlayerStats" + str(index+startYear_validation) + ".csv"
#     teamstats_validation = "Team_data\TeamStats" + str(index+startYear_validation) + ".csv"
#     player_list_validation[index] = pd.read_csv(playerstats_validation).values
#     winrate_list_validation[index] = pd.read_csv(teamstats_validation).values
#     data_list_validation[index] = player_list_validation[index][:,4:]
#     data_list_validation[index] = data_norm(data_list_validation[index])
#     team_idx_validation[index] = make_idx(winrate_list_validation[index],Abbr,teamName)
#     data_input_validation[index] = make_team(player_list_validation[index],data_list_validation[index],team_idx_validation[index],str(index+startYear_validation),device,dtype,need_idx)
#     loader_validation = dict(loader_validation,**data_input_validation[index])
# '''

In [36]:
'''
if the raw file is labeled with team,then use code in this cell, if not,use next cell
'''
player_list_test = [0] * testYear
winrate_list_test = [0] * testYear
data_list_test = [0] * testYear
team_idx_test = [0] * testYear
data_input_test = [0] * testYear
for index in range(testYear):
    playerstats_test = "Player_Data/AvgPlayerStats" + str(index+startYear_test) + ".csv"
    teamstats_test = "Team_Data/TeamStats" + str(index+startYear_test) + ".csv"
    
    player_list_test[index] = (pd.read_csv(playerstats_test)).values
    winrate_list_test[index] = (pd.read_csv(teamstats_test)).values
    data_list_test[index] = player_list_test[index][:,4:]
    data_list_test[index] = data_norm(data_list_test[index])
    team_idx_test[index] = make_idx(winrate_list_test[index],Abbr,teamName)
    data_input_test[index] = make_team(player_list_test[index],data_list_test[index],team_idx_test[index],str(index+startYear_test),device,dtype,need_idx)
#print(data_input_test[0]['BOS2018'])
#train
#loader_train

In [37]:
# change the file name
playerstats_test = "Player_Data/AvgPlayerStats" + str(index+startYear_test) + ".csv"
teamstats_test = "Team_Data/TeamStats" + str(index+startYear_test) + ".csv"
player_list_test = (pd.read_csv(playerstats_test)).values
winrate_list_test = (pd.read_csv(teamstats_test)).values
# 4 could be changed, it correspond to the column of played minutes per season
data_list_test = player_list_test[:,4:]
data_list_test = data_norm(data_list_test)
#data_input_test = make_input(data_list_test)

In [38]:
#load the existing model
model = torch.load('Trained_Model/index_code1_poly2.2.fit')
model.state_dict()

OrderedDict([('0.0.weight',
              tensor([[ 0.2331,  0.0535, -0.0071,  0.0219, -0.0711, -0.1629,  0.2112,  0.0138,
                       -0.0278,  0.0070, -0.0656, -0.1519, -0.0515, -0.0335, -0.0185,  0.0774,
                       -0.0806, -0.0046,  0.0327, -0.0025,  0.0066, -0.0292,  0.0910, -0.0068,
                        0.0370, -0.0357,  0.0239,  0.0836,  0.0815,  0.1201,  0.1651, -0.1435]]))])

In [33]:
#see the effect on testing set
from matplotlib import pyplot as plt
x = []
y = []#test
loss = 0
# change the data_input_test[0] for validation is ok to see performance
# change data_input_test[0] to your real input(if one team, no for loop)
for t, content in data_input_test[0].items():

    PER_C = model(content[0])
            # Loss is the mean square error
    win_rate = sum(PER_C)
    plt.scatter(win_rate[0].data,content[1][0].data)
    print(win_rate,content[1])
    x.append(win_rate[0].data)
    y.append(content[1][0].data)
    loss += (win_rate[0]-content[1][0].data)**2
print(loss)
z1 = np.polyfit(x, y, 1)
p1 = np.poly1d(z1)
x=np.linspace(0,1,100)
y = z1[0]*x+z1[1]

l1 = plt.plot(x,y,color='blue',linewidth=1.0,label='line')
x=np.linspace(0,1,100)
y =x
l2 = plt.plot(x,y,color='red',linewidth=1.0,linestyle='--',label='line')
plt.legend(labels = ['curve_line', 'y=x'], loc = 'best')
plt.show()

tensor([0.5890], grad_fn=<ThAddBackward>) tensor(0.6710)
tensor([0.6423], grad_fn=<ThAddBackward>) tensor(0.6340)
tensor([0.3717], grad_fn=<ThAddBackward>) tensor(0.3540)
tensor([0.5077], grad_fn=<ThAddBackward>) tensor(0.6100)
tensor([0.5949], grad_fn=<ThAddBackward>) tensor(0.5850)
tensor([0.4613], grad_fn=<ThAddBackward>) tensor(0.5370)
tensor([0.5169], grad_fn=<ThAddBackward>) tensor(0.4760)
tensor([0.2854], grad_fn=<ThAddBackward>) tensor(0.3290)
tensor([0.5419], grad_fn=<ThAddBackward>) tensor(0.5370)
tensor([0.5039], grad_fn=<ThAddBackward>) tensor(0.5240)
tensor([0.3416], grad_fn=<ThAddBackward>) tensor(0.3050)
tensor([0.3153], grad_fn=<ThAddBackward>) tensor(0.2930)
tensor([0.6107], grad_fn=<ThAddBackward>) tensor(0.5980)
tensor([0.6765], grad_fn=<ThAddBackward>) tensor(0.5850)
tensor([0.6801], grad_fn=<ThAddBackward>) tensor(0.5850)
tensor([0.6306], grad_fn=<ThAddBackward>) tensor(0.5730)
tensor([0.5976], grad_fn=<ThAddBackward>) tensor(0.5610)
tensor([0.7160], grad_fn=<ThAdd

/Users/lwx8999/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]
/Users/lwx8999/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()
/Users/lwx8999/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


<Figure size 640x480 with 1 Axes>